In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2


2026-02-05 17:19:13.527792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('coffee_beans.csv')
X = df["filepaths"]
y = df["class index"]


In [3]:
def load_images(x):
    images = []
    for path in x:
        img = cv2.imread(path)
        if img is None:
            print(f"Could not read image: {path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # normalize
    
        images.append(img)
    
    images = np.array(images, dtype=np.float32)
    return images



Split dataset into training, validation and test set. This split will be used to choose the best neural network architecture for the final model.

In [4]:
from sklearn.model_selection import train_test_split

X_train, x_, y_train, y_ = train_test_split(X , y, test_size=0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(x_ , y_, test_size=0.20, random_state=1)

del x_, y_

Load the images defined in the $x$ sets.

In [5]:
X_train = load_images(X_train)
X_cv = load_images(X_cv)
X_test = load_images(X_test)

In [8]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2   
from tensorflow.keras import layers, models

tf.keras.utils.set_random_seed(42) # seed for reproducibility

# Pre Trained CNN
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze pretrained weights

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(4, activation='linear')
])

nn_models = [model]

In [9]:
from sklearn.metrics import log_loss

nn_train_cross_entropy = []
nn_cv_cross_entropy = []

for model in nn_models:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), # gradient descent optimatiation
        loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'], # ??? have to research
    )
    print(f"Training {model.name}...")
    
    model.fit(X_train, y_train, epochs=10)
    print("Done\n")

    # Instantiate loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    # Record the training Log Losses
    yhat = model.predict(X_train)
    # pred_class = np.argmax(yhat, axis=1)[0]
    train_cross_entropy = loss_fn(y_train, yhat).numpy()
    nn_train_cross_entropy.append(train_cross_entropy)

    # Record the cross validation Log Losses
    yhat = model.predict(X_cv)
    # pred_class = np.argmax(yhat, axis=1)[0]
    cv_cross_entropy = loss_fn(y_cv, yhat).numpy()
    nn_cv_cross_entropy.append(cv_cross_entropy)

Training sequential_1...
Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 25s 674ms/step - accuracy: 0.8823 - loss: 1.1651
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 19s 635ms/step - accuracy: 0.9563 - loss: 0.7511
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 629ms/step - accuracy: 0.9615 - loss: 0.4477
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 21s 704ms/step - accuracy: 0.9792 - loss: 0.5493
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 668ms/step - accuracy: 0.9875 - loss: 0.2512
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 650ms/step - accuracy: 0.9781 - loss: 0.3299
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 652ms/step - accuracy: 0.9854 - loss: 0.3147
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 19s 648ms/step - accuracy: 0.9781 - loss: 0.3852
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 677ms/step - accuracy: 0.9760 - loss: 0.6257
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 654ms/step - accuracy: 0.9854 - loss: 0.5006
Done

30/30 ━━━━━━━━━━━━━━━━━━━━ 20s 650ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 669ms/step


Because softmax is integrated into the output layer the output s a vector of probabilities.

In [16]:
for model in nn_models:
    print(
        f"{model.name} Performance Summary\n"
        f"-----------------------------\n"
        f"Training Cross-Entropy Loss : {nn_train_cross_entropy[-1]:.4f}\n"
        f"CV Cross-Entropy Loss       : {nn_cv_cross_entropy[-1]:.4f}\n"
    )


sequential_1 Performance Summary
-----------------------------
Training Cross-Entropy Loss : 0.0252
CV Cross-Entropy Loss       : 1.1881



Make predictions

In [10]:
class_names = {
    0: "Dark",
    1: "Green",
    2: "Light",
    3: "Medium"
}


In [11]:
IMG_SIZE = 224

img = cv2.imread("test/Dark/dark (14).png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0

img = np.expand_dims(img, axis=0)  # shape (1, 224, 224, 3)

pred = model.predict(img)
pred_class = np.argmax(pred, axis=1)[0]

print("Predicted:", class_names[pred_class])
print("Class probabilities (Dark, Green, Light, Medium):", pred[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted: Dark
Class probabilities (Dark, Green, Light, Medium): [ 176.53752  -165.09375  -241.84581   -25.023415]


Save the model

In [12]:
model.save("coffee_roast_model.keras")